<a href="https://colab.research.google.com/github/donganzh/csc413/blob/main/FinalProject/Team88_Drake_rap_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#A Comparison of Transformer and LSTM Towards Rap Lyrics Generation Inspired by Drake’s Music

# Data Processing

First we import the drake lyrics csv file and extract the lyrics column to construct a new text file containing only the lyrics. The data set is available at [kaggle](https://www.kaggle.com/juicobowley/drake-lyrics).

In [ ]:
import pandas as pd
df = pd.read_csv('./drake_data.csv',skiprows=1, names=['album', 'title', 'url', 'lyrics', 'review'])
df

,album,title,url,lyrics,review
0,Certified Lover Boy,Certified Lover Boy* Lyrics,https://genius.com/Drake-certified-lover-boy-l...,[Verse]\nPut my feelings on ice\nAlways been a...,8.7K
1,Certified Lover Boy,Like I’m Supposed To/Do Things Lyrics,https://genius.com/Drake-like-im-supposed-to-d...,[Verse]\nHands are tied\nSomeone's in my ear f...,38.8K
2,Certified Lover Boy,Not Around Lyrics,https://genius.com/Drake-not-around-lyrics,"[Intro]\nYeah, we back\nWassup ladies?\nSwisha...",129.8K
3,Certified Lover Boy,In the Cut (Ft. Roddy Ricch) Lyrics,https://genius.com/Drake-in-the-cut-lyrics,"[Intro: Drake]\nAyy, yeah\nPipe this shit up a...",72.1K
4,Certified Lover Boy,Zodiac Sign (Ft. Jessie Reyez) Lyrics,https://genius.com/Drake-zodiac-sign-lyrics,[Verse 1: Drake]\nYou ask how many girls I bee...,54.8K
...,...,...,...,...,...
285,Unreleased Songs,Pop Style (Demo) (Ft. Kanye West) Lyrics,https://genius.com/Drake-pop-style-demo-lyrics,Lyrics from cut studio recording\n\n[Verse 1: ...,NaN
286,Unreleased Songs,Time Flies (Demo) (Ft. Future) Lyrics,https://genius.com/Drake-time-flies-demo-lyrics,Lyrics from Snippet\n\n[Chorus: Drake & Future...,NaN
287,Unreleased Songs,Walk It Talk It (Demo) by Quavo (Ft. Drake) Ly...,https://genius.com/Quavo-walk-it-talk-it-demo-...,"[Intro: Quavo]\nYeah, yeah\n(Deko)\nWoah, hold...",NaN
288,Unreleased Songs,God’s Plan (Demo) (Ft. Trippie Redd) Lyrics,https://genius.com/Drake-gods-plan-demo-lyrics,"[Intro: Drake]\nThey wishing, they wishing, th...",NaN


In [ ]:
def build_dataset(df, dest_path):
    f = open(dest_path, 'w')
    data = ''
    lyrics = df['lyrics'].tolist()
    for lyric in lyrics:
        lyric = str(lyric).strip()
        data+=lyric
    f.write(data)
    f.close()

In [ ]:
import re
build_dataset(df, './drake_data.txt')

#part1 - GPT2

We adapt the transformer models from the paper ["Language Models are Unsupervised Multitask Learners"](https://d4mucfpksywv.cloudfront.net/better-language-models/language-models.pdf). This model was developed by Radford, Alec and Wu, Jeff and Child, Rewon and Luan, David and Amodei, Dario and Sutskever, Ilya in 2019 and is available at [this repo](https://github.com/openai/gpt-2).

We fine tone the model by providing different step sizes.




## Model

In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
gpt2.download_gpt2(model_name="124M")

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Fetching checkpoint: 1.05Mit [00:00, 456Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 1.28Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 232Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:43, 11.5Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 688Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 1.68Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 1.40Mit/s]


In [ ]:
gpt2.mount_gdrive()

Mounted at /content/drive


In [ ]:
filename = "./drake_data.txt"

## Training

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.finetune(sess, 
             dataset = filename,
             model_name = "124M",
             steps = 1000,
             restore_from = "fresh",
             run_name = 'run1',
             print_every = 10,
             sample_every =200,
             save_every =500)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:01<00:00,  1.27s/it]


dataset has 226068 tokens
Training...
[10 | 15.30] loss=3.05 avg=3.05
[20 | 23.27] loss=3.16 avg=3.11
[30 | 31.22] loss=2.66 avg=2.96
[40 | 39.20] loss=2.65 avg=2.88
[50 | 47.23] loss=2.51 avg=2.80
[60 | 55.22] loss=2.56 avg=2.76
[70 | 63.22] loss=2.41 avg=2.71
[80 | 71.18] loss=2.55 avg=2.69
[90 | 79.15] loss=2.78 avg=2.70
[100 | 87.12] loss=2.44 avg=2.67
[110 | 95.15] loss=2.54 avg=2.66
[120 | 103.12] loss=2.36 avg=2.63
[130 | 111.11] loss=2.66 avg=2.64
[140 | 119.05] loss=2.37 avg=2.62
[150 | 127.09] loss=2.49 avg=2.61
[160 | 135.04] loss=2.10 avg=2.57
[170 | 143.03] loss=2.48 avg=2.57
[180 | 151.03] loss=1.67 avg=2.51
[190 | 159.01] loss=1.67 avg=2.46
[200 | 167.02] loss=1.89 avg=2.43
======== SAMPLE 1 ========
 than a season pass
I should probably think about it, you know what it is

[Chorus]
So- and-so, that's my baby girl
She cryin' and I cryin' and I'ma get her
She cryin', I know how to get her, but I can't get her to you

[Verse 3]
Your mother is in the bathroom, I'm comin' th

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

## Sampling

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

In [ ]:
gpt2.generate(sess, run_name='run1')

#part 2 - LSTM

We adapt the TensorFlow implementation of Andrej Karpathy's [Char-RNN](https://github.com/karpathy/char-rnn). This code implements multi-layer Recurrent Neural Network (RNN, LSTM, and GRU) for training/sampling from character-level language models. Please refer to his blog post [The Unreasonable Effectiveness of Recurrent Neural Network](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) to learn more about this model. This TensorFlow model is developed by Chen Liang from Google Brain and is available at [this repo](https://github.com/crazydonkey200/tensorflow-char-rnn)

We fine tone the model using the following hyperparameters:


1.   num of LSTM parameters
2.   num of hidden layers
3.   batch size
4.   learning rate

After 30 epochs, the training loss becomes stable and produces a reasonable model for text generation. 

## 1. Model

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import logging
import time
import numpy as np
import tensorflow as tf
import json
import shutil
import sys
from six import iteritems

class CharRNN(object):
  """Character RNN model."""
  
  def __init__(self, is_training, batch_size, num_unrollings, vocab_size, 
               hidden_size, max_grad_norm, embedding_size, num_layers,
               learning_rate, dropout=0.0, input_dropout=0.0, use_batch=True):
    self.batch_size = batch_size
    self.num_unrollings = num_unrollings
    if not use_batch:
      self.batch_size = 1
      self.num_unrollings = 1
    self.hidden_size = hidden_size
    self.vocab_size = vocab_size
    self.max_grad_norm = max_grad_norm
    self.num_layers = num_layers
    self.embedding_size = embedding_size
    self.model = 'lstm'
    self.dropout = dropout
    self.input_dropout = input_dropout
    if embedding_size <= 0:
      self.input_size = vocab_size
      # Don't do dropout on one hot representation.
      self.input_dropout = 0.0
    else:
      self.input_size = embedding_size
    self.model_size = (embedding_size * vocab_size + # embedding parameters
                       # lstm parameters
                       4 * hidden_size * (hidden_size + self.input_size + 1) +
                       # softmax parameters
                       vocab_size * (hidden_size + 1) +
                       # multilayer lstm parameters for extra layers.
                       (num_layers - 1) * 4 * hidden_size *
                       (hidden_size + hidden_size + 1))
    # self.decay_rate = decay_rate

    # Placeholder to feed in input and targets/labels data.
    self.input_data = tf.compat.v1.placeholder(tf.int64,
                                     [self.batch_size, self.num_unrollings],
                                     name='inputs')
    self.targets = tf.compat.v1.placeholder(tf.int64,
                                  [self.batch_size, self.num_unrollings],
                                  name='targets')


    cell_fn = tf.compat.v1.nn.rnn_cell.BasicLSTMCell

    # params = {'input_size': self.input_size}
    params = {}
    
    # add bias to forget gate in lstm.
    params['forget_bias'] = 0.0
    params['state_is_tuple'] = True
    # Create multilayer cell.
    cell = cell_fn(
        self.hidden_size, reuse=tf.compat.v1.get_variable_scope().reuse,
        **params)

    cells = [cell]

    for i in range(self.num_layers-1):
      higher_layer_cell = cell_fn(
          self.hidden_size, reuse=tf.compat.v1.get_variable_scope().reuse,
          **params)
      cells.append(higher_layer_cell)

    if is_training and self.dropout > 0:
      cells = [tf.contrib.rnn.DropoutWrapper(
        cell,
        output_keep_prob=1.0-self.dropout)
               for cell in cells]

    multi_cell = tf.compat.v1.nn.rnn_cell.MultiRNNCell(cells)

    with tf.compat.v1.name_scope('initial_state'):
      # zero_state is used to compute the intial state for cell.
      self.zero_state = multi_cell.zero_state(self.batch_size, tf.float32)

      self.initial_state = create_tuple_placeholders_with_default(
        multi_cell.zero_state(batch_size, tf.float32),
        extra_dims=(None,),
        shape=multi_cell.state_size)      
      

    # Embeddings layers.
    with tf.compat.v1.name_scope('embedding_layer'):
      if embedding_size > 0:
        self.embedding = tf.compat.v1.get_variable(
          'embedding', [self.vocab_size, self.embedding_size])
      else:
        self.embedding = tf.constant(np.eye(self.vocab_size), dtype=tf.float32)

      inputs = tf.nn.embedding_lookup(self.embedding, self.input_data)
      if is_training and self.input_dropout > 0:
        inputs = tf.nn.dropout(inputs, 1 - self.input_dropout)

    with tf.compat.v1.name_scope('slice_inputs'):
      # Slice inputs into a list of shape [batch_size, 1] data colums.
      sliced_inputs = [tf.squeeze(input_, [1])
                       for input_ in tf.split(axis=1, num_or_size_splits=self.num_unrollings, value=inputs)]
      
    # Copy cell to do unrolling and collect outputs.
    outputs, final_state = tf.compat.v1.nn.static_rnn(
      multi_cell, sliced_inputs,
      initial_state=self.initial_state)

    self.final_state = final_state

    with tf.compat.v1.name_scope('flatten_ouputs'):
      # Flatten the outputs into one dimension.
      flat_outputs = tf.reshape(tf.concat(axis=1, values=outputs), [-1, hidden_size])

    with tf.compat.v1.name_scope('flatten_targets'):
      # Flatten the targets too.
      flat_targets = tf.reshape(tf.concat(axis=1, values=self.targets), [-1])
    
    # Create softmax parameters, weights and bias.
    with tf.compat.v1.name_scope('softmax') as sm_vs:
      softmax_w = tf.compat.v1.get_variable("softmax_w", [hidden_size, vocab_size])
      softmax_b = tf.compat.v1.get_variable("softmax_b", [vocab_size])
      self.logits = tf.matmul(flat_outputs, softmax_w) + softmax_b
      self.probs = tf.nn.softmax(self.logits)

    with tf.compat.v1.name_scope('loss'):
      # Compute mean cross entropy loss for each output.
      loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=self.logits, labels=flat_targets)
      self.mean_loss = tf.reduce_mean(loss)

    with tf.compat.v1.name_scope('loss_monitor'):
      # Count the number of elements and the sum of mean_loss
      # from each batch to compute the average loss.
      count = tf.Variable(1.0, name='count')
      sum_mean_loss = tf.Variable(1.0, name='sum_mean_loss')
      
      self.reset_loss_monitor = tf.group(sum_mean_loss.assign(0.0),
                                         count.assign(0.0),
                                         name='reset_loss_monitor')
      self.update_loss_monitor = tf.group(sum_mean_loss.assign(sum_mean_loss +
                                                               self.mean_loss),
                                          count.assign(count + 1),
                                          name='update_loss_monitor')
      with tf.control_dependencies([self.update_loss_monitor]):
        self.average_loss = sum_mean_loss / count
        self.ppl = tf.exp(self.average_loss)

      # Monitor the loss.
      loss_summary_name = "average_loss"
      ppl_summary_name = "perplexity"
  
      average_loss_summary = tf.compat.v1.summary.scalar(loss_summary_name, self.average_loss)
      ppl_summary = tf.compat.v1.summary.scalar(ppl_summary_name, self.ppl)

    # Monitor the loss.
    self.summaries = tf.compat.v1.summary.merge([average_loss_summary, ppl_summary],
                                      name='loss_monitor')
    
    self.global_step = tf.compat.v1.get_variable('global_step', [],
                                       initializer=tf.compat.v1.constant_initializer(0.0))

    self.learning_rate = tf.constant(learning_rate)
    if is_training:
      tvars = tf.compat.v1.trainable_variables()
      grads, _ = tf.clip_by_global_norm(tf.gradients(self.mean_loss, tvars),
                                        self.max_grad_norm)

      optimizer = tf.compat.v1.train.AdamOptimizer(self.learning_rate)

      self.train_op = optimizer.apply_gradients(zip(grads, tvars),
                                                global_step=self.global_step)
      
  def run_epoch(self, session, data_size, batch_generator, is_training,
                verbose=0, freq=10, summary_writer=None, debug=False, divide_by_n=1):
    """Runs the model on the given data for one full pass."""
    # epoch_size = ((data_size // self.batch_size) - 1) // self.num_unrollings
    epoch_size = data_size // (self.batch_size * self.num_unrollings)
    if data_size % (self.batch_size * self.num_unrollings) != 0:
        epoch_size += 1

    if verbose > 0:
        logging.info('epoch_size: %d', epoch_size)
        logging.info('data_size: %d', data_size)
        logging.info('num_unrollings: %d', self.num_unrollings)
        logging.info('batch_size: %d', self.batch_size)

    if is_training:
      extra_op = self.train_op
    else:
      extra_op = tf.no_op()

    # Prepare initial state and reset the average loss
    # computation.
    state = session.run(self.zero_state)
    self.reset_loss_monitor.run()
    start_time = time.time()
    for step in range(epoch_size // divide_by_n):
      # Generate the batch and use [:-1] as inputs and [1:] as targets.
      data = batch_generator.next()
      inputs = np.array(data[:-1]).transpose()
      targets = np.array(data[1:]).transpose()

      ops = [self.average_loss, self.final_state, extra_op,
             self.summaries, self.global_step, self.learning_rate]

      feed_dict = {self.input_data: inputs, self.targets: targets,
                   self.initial_state: state}

      results = session.run(ops, feed_dict)
      average_loss, state, _, summary_str, global_step, lr = results
      
      ppl = np.exp(average_loss)
      if (verbose > 0) and ((step+1) % freq == 0):
        logging.info("%.1f%%, step:%d, perplexity: %.3f, speed: %.0f words",
                     (step + 1) * 1.0 / epoch_size * 100, step, ppl,
                     (step + 1) * self.batch_size * self.num_unrollings /
                     (time.time() - start_time))

    logging.info("Perplexity: %.3f, speed: %.0f words per sec",
                 ppl, (step + 1) * self.batch_size * self.num_unrollings /
                 (time.time() - start_time))
    return ppl, summary_str, global_step

  def sample_seq(self, session, length, start_text, vocab_index_dict,
                 index_vocab_dict, temperature=1.0, max_prob=True):

    state = session.run(self.zero_state)

    # use start_text to warm up the RNN.
    if start_text is not None and len(start_text) > 0:
      seq = list(start_text)
      for char in start_text[:-1]:
        x = np.array([[char2id(char, vocab_index_dict)]])
        state = session.run(self.final_state,
                            {self.input_data: x,
                             self.initial_state: state})
      x = np.array([[char2id(start_text[-1], vocab_index_dict)]])
    else:
      vocab_size = len(vocab_index_dict.keys())
      x = np.array([[np.random.randint(0, vocab_size)]])
      seq = []

    for i in range(length):
      state, logits = session.run([self.final_state,
                                   self.logits],
                                  {self.input_data: x,
                                   self.initial_state: state})
      unnormalized_probs = np.exp((logits - np.max(logits)) / temperature)
      probs = unnormalized_probs / np.sum(unnormalized_probs)

      if max_prob:
        sample = np.argmax(probs[0])
      else:
        sample = np.random.choice(self.vocab_size, 1, p=probs[0])[0]

      seq.append(id2char(sample, index_vocab_dict))
      x = np.array([[sample]])
    return ''.join(seq)
      
        
class BatchGenerator(object):
    """Generate and hold batches."""
    def __init__(self, text, batch_size, n_unrollings, vocab_size,
                 vocab_index_dict, index_vocab_dict):
      self._text = text
      self._text_size = len(text)
      self._batch_size = batch_size
      self.vocab_size = vocab_size
      self._n_unrollings = n_unrollings
      self.vocab_index_dict = vocab_index_dict
      self.index_vocab_dict = index_vocab_dict
      
      segment = self._text_size // batch_size

      # number of elements in cursor list is the same as
      # batch_size.  each batch is just the collection of
      # elements in where the cursors are pointing to.
      self._cursor = [ offset * segment for offset in range(batch_size)]
      self._last_batch = self._next_batch()
      
    def _next_batch(self):
      """Generate a single batch from the current cursor position in the data."""
      batch = np.zeros(shape=(self._batch_size), dtype=np.float)
      for b in range(self._batch_size):
        batch[b] = char2id(self._text[self._cursor[b]], self.vocab_index_dict)
        self._cursor[b] = (self._cursor[b] + 1) % self._text_size
      return batch

    def next(self):
      """Generate the next array of batches from the data. The array consists of
      the last batch of the previous array, followed by num_unrollings new ones.
      """
      batches = [self._last_batch]
      for step in range(self._n_unrollings):
        batches.append(self._next_batch())
      self._last_batch = batches[-1]
      return batches


# Utility functions
def batches2string(batches, index_vocab_dict):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, id2char_list(b, index_vocab_dict))]
  return s


def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]


def char2id(char, vocab_index_dict):
  try:
    return vocab_index_dict[char]
  except KeyError:
    logging.info('Unexpected char %s', char)
    return 0


def id2char(index, index_vocab_dict):
  return index_vocab_dict[index]

    
def id2char_list(lst, index_vocab_dict):
  return [id2char(i, index_vocab_dict) for i in lst]


def create_tuple_placeholders_with_default(inputs, extra_dims, shape):
  if isinstance(shape, int):
    result = tf.compat.v1.placeholder_with_default(
      inputs, list(extra_dims) + [shape])
  else:
    subplaceholders = [create_tuple_placeholders_with_default(
      subinputs, extra_dims, subshape)
                       for subinputs, subshape in zip(inputs, shape)]
    t = type(shape)
    if t == tuple:
      result = t(subplaceholders)
    else:
      result = t(*subplaceholders)    
  return result

        
def create_tuple_placeholders(dtype, extra_dims, shape):
  if isinstance(shape, int):
    result = tf.compat.v1.placeholder(dtype, list(extra_dims) + [shape])
  else:
    subplaceholders = [create_tuple_placeholders(dtype, extra_dims, subshape)
                       for subshape in shape]
    t = type(shape)

    # Handles both tuple and LSTMStateTuple.
    if t == tuple:
      result = t(subplaceholders)
    else:
      result = t(*subplaceholders)
  return result
  


##2. training

In [ ]:
def train(data_file, num_epochs, hidden_size, num_layers, batch_size, output_dir):

    encoding = 'utf-8'
    n_save=1
    max_to_keep=5
    embedding_size=0
    num_unrollings=10
    train_frac=0.8
    valid_frac=0.1
    dropout=0.0

    input_dropout=0.0

    # Parameters for gradient descent.
    max_grad_norm=5
    learning_rate=1e-4
    decay_rate=0.95

    # Parameters for logging.    
    progress_freq=100
    verbose=0

    # Parameters to feed in the initial model and current best model.
    init_model=''
    best_model=''
    best_valid_ppl=np.Inf
                       
    # Parameters for using saved best models.
    init_dir=''

    # Specifying location to store model, best model and tensorboard log.
    save_model = os.path.join(output_dir, 'save_model/model')
    save_best_model = os.path.join(output_dir, 'best_model/model')
    tb_log_dir = os.path.join(output_dir, 'tensorboard_log/')
    vocab_file = ''

    # Create necessary directories.
    if init_dir:
        output_dir = init_dir
    else:
        if os.path.exists(output_dir):
            shutil.rmtree(output_dir)
        for paths in [save_model, save_best_model,
                      tb_log_dir]:
            os.makedirs(os.path.dirname(paths))


    logging.basicConfig(stream=sys.stdout,
                        format='%(asctime)s %(levelname)s:%(message)s', 
                        level=logging.INFO,
                        datefmt='%I:%M:%S')

    print('=' * 60)
    print('All final and intermediate outputs will be stored in %s/' % output_dir)
    print('=' * 60 + '\n')

    # Prepare parameters.
    if init_dir:
        with open(os.path.join(init_dir, 'result.json'), 'r') as f:
            result = json.load(f)
        params = result['params']
        init_model = result['latest_model']
        best_model = result['best_model']
        best_valid_ppl = result['best_valid_ppl']
        if 'encoding' in result:
            encoding = result['encoding']
        else:
            encoding = 'utf-8'
        vocab_file = os.path.join(init_dir, 'vocab.json')
    else:
        params = {'batch_size': batch_size,
                  'num_unrollings': num_unrollings,
                  'hidden_size': hidden_size,
                  'max_grad_norm': max_grad_norm,
                  'embedding_size': embedding_size,
                  'num_layers': num_layers,
                  'learning_rate': learning_rate,
                  'dropout': dropout,
                  'input_dropout': input_dropout}
        best_model = ''
    logging.info('Parameters are:\n%s\n', json.dumps(params, sort_keys=True, indent=4))

    # Read and split data.
    print('Reading data from: %s', data_file)
    with codecs.open(data_file, 'r', encoding=encoding) as f:
        text = f.read()

    logging.info('Number of characters: %s', len(text))

    logging.info('Creating train, valid, test split')
    train_size = int(train_frac * len(text))
    valid_size = int(valid_frac * len(text))
    test_size = len(text) - train_size - valid_size
    train_text = text[:train_size]
    valid_text = text[train_size:train_size + valid_size]
    test_text = text[train_size + valid_size:]

    if vocab_file:
        vocab_index_dict, index_vocab_dict, vocab_size = load_vocab(
          vocab_file, encoding)
    else:
        logging.info('Creating vocabulary')
        vocab_index_dict, index_vocab_dict, vocab_size = create_vocab(text)
        vocab_file = os.path.join(output_dir, 'vocab.json')
        save_vocab(vocab_index_dict, vocab_file, encoding)
        logging.info('Vocabulary is saved in %s', vocab_file)
        vocab_file = vocab_file

    params['vocab_size'] = vocab_size
    logging.info('Vocab size: %d', vocab_size)

    # Create batch generators.
    batch_size = params['batch_size']
    num_unrollings = params['num_unrollings']
    train_batches = BatchGenerator(train_text, batch_size, num_unrollings, vocab_size, 
                                   vocab_index_dict, index_vocab_dict)
    valid_batches = BatchGenerator(valid_text, batch_size, num_unrollings, vocab_size,
                                   vocab_index_dict, index_vocab_dict)
    test_batches = BatchGenerator(test_text, 1, 1, vocab_size,
                                  vocab_index_dict, index_vocab_dict)
        
    # Create graphs
    graph = tf.Graph()
    with graph.as_default():
        with tf.compat.v1.name_scope('training'):
            train_model = CharRNN(is_training=True, use_batch=True, **params)
        tf.compat.v1.get_variable_scope().reuse_variables()
        with tf.compat.v1.name_scope('validation'):
            valid_model = CharRNN(is_training=False, use_batch=True, **params)
        with tf.compat.v1.name_scope('evaluation'):
            test_model = CharRNN(is_training=False, use_batch=False, **params)
            saver = tf.compat.v1.train.Saver(name='checkpoint_saver', max_to_keep=max_to_keep)
            best_model_saver = tf.compat.v1.train.Saver(name='best_model_saver')

    logging.info('Model size (number of parameters): %s\n', train_model.model_size)
    logging.info('Start training\n')

    result = {}
    result['params'] = params
    result['vocab_file'] = vocab_file
    result['encoding'] = encoding

    try:
        with tf.compat.v1.Session(graph=graph) as session:
            graph_info = session.graph

            train_writer = tf.compat.v1.summary.FileWriter(tb_log_dir + 'train/', graph_info)
            valid_writer = tf.compat.v1.summary.FileWriter(tb_log_dir + 'valid/', graph_info)

            # load a saved model or start from random initialization.
            if init_model:
                saver.restore(session, init_model)
            else:
                tf.compat.v1.global_variables_initializer().run()
            for i in range(num_epochs):
                for j in range(n_save):
                    logging.info(
                        '=' * 19 + ' Epoch %d: %d/%d' + '=' * 19 + '\n', i+1, j+1, n_save)
                    logging.info('Training on training set')
                    # training step
                    ppl, train_summary_str, global_step = train_model.run_epoch(
                        session,
                        train_size,
                        train_batches,
                        is_training=True,
                        verbose=verbose,
                        freq=progress_freq,
                        divide_by_n=n_save)
                    # record the summary
                    train_writer.add_summary(train_summary_str, global_step)
                    train_writer.flush()
                    # save model
                    saved_path = saver.save(session, save_model,
                                            global_step=train_model.global_step)
                    logging.info('Latest model saved in %s\n', saved_path)
                    logging.info('Evaluate on validation set')

                    # valid_ppl, valid_summary_str, _ = valid_model.run_epoch(
                    valid_ppl, valid_summary_str, _ = valid_model.run_epoch(
                        session,
                        valid_size,
                        valid_batches, 
                        is_training=False,
                        verbose=verbose,
                        freq=progress_freq)

                    # save and update best model
                    if (not best_model) or (valid_ppl < best_valid_ppl):
                        best_model = best_model_saver.save(
                            session,
                            save_best_model,
                            global_step=train_model.global_step)
                        best_valid_ppl = valid_ppl
                    valid_writer.add_summary(valid_summary_str, global_step)
                    valid_writer.flush()
                    logging.info('Best model is saved in %s', best_model)
                    logging.info('Best validation ppl is %f\n', best_valid_ppl)
                    result['latest_model'] = saved_path
                    result['best_model'] = best_model
                    # Convert to float because numpy.float is not json serializable.
                    result['best_valid_ppl'] = float(best_valid_ppl)
                    result_path = os.path.join(output_dir, 'result.json')
                    if os.path.exists(result_path):
                        os.remove(result_path)
                    with open(result_path, 'w') as f:
                        json.dump(result, f, indent=2, sort_keys=True)

            logging.info('Latest model is saved in %s', saved_path)
            logging.info('Best model is saved in %s', best_model)
            logging.info('Best validation ppl is %f\n', best_valid_ppl)
            logging.info('Evaluate the best model on test set')
            saver.restore(session, best_model)
            test_ppl, _, _ = test_model.run_epoch(session, test_size, test_batches,
                                                   is_training=False,
                                                   verbose=verbose,
                                                   freq=progress_freq)
            result['test_ppl'] = float(test_ppl)
    finally:
        result_path = os.path.join(output_dir, 'result.json')
        if os.path.exists(result_path):
            os.remove(result_path)
        with open(result_path, 'w') as f:
            json.dump(result, f, indent=2, sort_keys=True)


def create_vocab(text):
    unique_chars = list(set(text))
    vocab_size = len(unique_chars)
    vocab_index_dict = {}
    index_vocab_dict = {}
    for i, char in enumerate(unique_chars):
        vocab_index_dict[char] = i
        index_vocab_dict[i] = char
    return vocab_index_dict, index_vocab_dict, vocab_size


def load_vocab(vocab_file, encoding):
    with codecs.open(vocab_file, 'r', encoding=encoding) as f:
        vocab_index_dict = json.load(f)
    index_vocab_dict = {}
    vocab_size = 0
    for char, index in iteritems(vocab_index_dict):
        index_vocab_dict[index] = char
        vocab_size += 1
    return vocab_index_dict, index_vocab_dict, vocab_size


def save_vocab(vocab_index_dict, vocab_file, encoding):
    with codecs.open(vocab_file, 'w', encoding=encoding) as f:
        json.dump(vocab_index_dict, f, indent=2, sort_keys=True)

##3. Sampling

In [ ]:
def sampling(init_dir, start_text, length):
    
    # Parameters for using saved best models.
    # Parameters for picking which model to use. 
    model_path=''
    # Parameters for sampling.
    temperature=1.0
    max_prob=False
    seed=999

    # Parameters for evaluation (computing perplexity of given text).
    evaluate=False
    example_text='The meaning of life is 42.'
    
    # Prepare parameters.
    with open(os.path.join(init_dir, 'result.json'), 'r') as f:
        result = json.load(f)
    params = result['params']

    if model_path:    
        best_model = model_path
    else:
        best_model = result['best_model']

    best_valid_ppl = result['best_valid_ppl']
    if 'encoding' in result:
        encoding = result['encoding']
    else:
        encoding = 'utf-8'
    vocab_file = os.path.join(init_dir, 'vocab.json')
    vocab_index_dict, index_vocab_dict, vocab_size = load_vocab(vocab_file, encoding)

    # Create graphs
    graph = tf.Graph()
    with graph.as_default():
        with tf.compat.v1.name_scope('evaluation'):
            test_model = CharRNN(is_training=False, use_batch=False, **params)
            saver = tf.compat.v1.train.Saver(name='checkpoint_saver')

    if evaluate:
        example_batches = BatchGenerator(example_text, 1, 1, vocab_size,
                                         vocab_index_dict, index_vocab_dict)
        with tf.compat.v1.Session(graph=graph) as session:
            saver.restore(session, best_model)
            ppl = test_model.run_epoch(session, len(example_text),
                                        example_batches,
                                        is_training=False)[0]
            print('Example text is: %s' % example_text)
            print('Perplexity is: %s' % ppl)
    else:
        if seed >= 0:
            np.random.seed(seed)
        # Sampling a sequence 
        with tf.compat.v1.Session(graph=graph) as session:
            saver.restore(session, best_model)
            sample = test_model.sample_seq(session, length, start_text,
                                            vocab_index_dict, index_vocab_dict,
                                            temperature=temperature,
                                            max_prob=max_prob)
            print('Sampled text is:\n%s' % sample)
        return sample

In [ ]:
train("./drake_data.txt", 30, 256, 4, 64, "./drake_output_hidden256_layers4_batch64")

All final and intermediate outputs will be stored in ./drake_output_hidden256_layers4_batch64/
All information will be logged to stdout

09:38:09 INFO:Parameters are:
{
    "batch_size": 64,
    "dropout": 0.0,
    "embedding_size": 0,
    "hidden_size": 256,
    "input_dropout": 0.0,
    "learning_rate": 0.0001,
    "max_grad_norm": 5,
    "num_layers": 4,
    "num_unrollings": 10
}

Reading data from: %s ./drake_data.txt
09:38:09 INFO:Number of characters: 769592
09:38:09 INFO:Creating train, valid, test split
09:38:09 INFO:Creating vocabulary
09:38:09 INFO:Vocabulary is saved in ./drake_output_hidden256_layers4_batch64/vocab.json
09:38:09 INFO:Vocab size: 104
09:38:09 WARNING:`tf.nn.rnn_cell.MultiRNNCell` is deprecated. This class is equivalent as `tf.keras.layers.StackedRNNCells`, and will be replaced by that in Tensorflow 2.0.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:702: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


09:38:11 WARNING:`tf.nn.rnn_cell.MultiRNNCell` is deprecated. This class is equivalent as `tf.keras.layers.StackedRNNCells`, and will be replaced by that in Tensorflow 2.0.
09:38:11 WARNING:`tf.nn.rnn_cell.MultiRNNCell` is deprecated. This class is equivalent as `tf.keras.layers.StackedRNNCells`, and will be replaced by that in Tensorflow 2.0.
09:38:11 INFO:Model size (number of parameters): 1972328

09:38:11 INFO:Start training

09:38:12 INFO:=================== Epoch 1: 1/1===================

09:38:12 INFO:Training on training set
09:38:23 INFO:Perplexity: 26.858, speed: 54570 words per sec
09:38:24 INFO:Latest model saved in ./drake_output_hidden256_layers4_batch64/save_model/model-962

09:38:24 INFO:Evaluate on validation set
09:38:25 INFO:Perplexity: 24.755, speed: 95755 words per sec
09:38:25 INFO:Best model is saved in ./drake_output_hidden256_layers4_batch64/best_model/model-962
09:38:25 INFO:Best validation ppl is 24.754810

09:38:25 INFO:=================== Epoch 2: 1/1=====

In [ ]:
sample_lyrics = sampling("./drake_output_hidden256_layers2_batch64", "we go up", 2000)

08:52:08 INFO:Creating graph
08:52:08 WARNING:`tf.nn.rnn_cell.MultiRNNCell` is deprecated. This class is equivalent as `tf.keras.layers.StackedRNNCells`, and will be replaced by that in Tensorflow 2.0.
INFO:tensorflow:Summary name average loss is illegal; using average_loss instead.
08:52:08 INFO:Summary name average loss is illegal; using average_loss instead.
INFO:tensorflow:Restoring parameters from ./drake_output_hidden256_layers2_batch64/best_model/model-3848
08:52:08 INFO:Restoring parameters from ./drake_output_hidden256_layers2_batch64/best_model/model-3848


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:702: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Sampled text is:
we go up?
Hold on, then
Ayy a ressice, "The next ineway" than I had up nellin'? 
[Chorus: Drake]
You won't abagh, a wings with my mole on the last night
Look, when it's going?

[Chorus]
Women, so bromistaat, Talls, Henn these-nigga
Lhink shots the ticket off with you, I'm all nextin' you
Estated-doxet you coupted on the grave like *08 cateron
My nehes tell it a, these niggas
I'm what at rega-game you faston

[Chorus]
Ufffom long
One conversation
She don't think that hass a talkin' I'm thinking that?
It's back
There's a man? (Trust)
You couldn't had to love you fills
Are still be ciniy's drink like a snve my best for alst to blunclessove it in when she bolming like I'm driven on when me like the y_ar kite for you
That's not now
My name on you (Jeffer all on but you baisted a conec
I was way and tyou, 'til you're doin' nigga

[Chorus: LiV Wayne]
Don't think to me, righ (brand ressected (Orill ard thingin' if you're back business
One more, nigga Only nigga she love

[Chor